# **To define a trader**

#### Stockait has an object called `Trader`, which is a key concept.
#### One trader includes a dataset and an artificial intelligence model, and performs model learning.
#### Stock trading (buy/sell) simulations are also available through traders.

### This file provides descriptions and examples of these `Traders`.

<br>

## **What Trader does**
#### Trader has three main functions.

![img](../image/trader_1.png)

#### Trader objects store 1) buy/sell information, artificial intelligence model information, 2) data sets, and additionally 3) name and label information.
#### We will look at these three functions one by one below, and look at them in more detail in an example of directly defining an object.

<br>

## **First, Trader object Definitions** 
#### Defining Trader / Buyer / Seller objects is a key function of Trader.

![img](../image/trader_2.png)

### **1)`trader.buyer`** 
Save the two types of objects in the buyer as a list. ([conditional_buyer, machine learning_buyer]).

#### **1.1)`conditional_buyer`** 
The object that determines the number of data sets as a filtering condition.

#### **1.2)`machinelearning_buyer`** 
An object that determines the number of times through the prediction probability of the machine learning model.

<br>

### **2)`trader.seller`** 
Saves the seller's object.

#### **2.1)`SubSeller`**
An object that sells all purchased stocks the next day.

<br> 

## **Second, Save datasets**

#### Within the trader object, data sets for model learning/verification/testing and stock trading are stored.

![img](../image/trader_3.png)

<br>

## **Third, Save Trader's Information**
#### It is a function that stores the `name` attribute to distinguish traders and the `label` attribute to put label information.

![img](../image/trader_4.png)

-------- 


<br>

#### Below, in order to take a closer look at the process of defining Trader, we define Trader as an example using the LightGBM model.
<br>

#### First, create an empty list to hold traders.

In [ ]:
lst_trader = [] 


<br>

### **Define ConditionalBuyer objects and sampling functions**
#### In Conditional Buyer, we directly define the function sampling that sets the conditions under which stock price data will be filtered. Conditions were created to use only datasets with a transaction value of 1 billion or more, excluding data with a closing rate of -0.05 to 0.05 with a closing rate of -0.05 to 0.05, which is too low.

In [ ]:
# conditional_buyer: Object that determines acquisition based on data filtering conditions 
b1_lg = sai.ConditionalBuyer()

def sampling1(df): # Create a conditional function
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) # Remove exceptions that exceed upper and lower limits
    condition2 = df.D0_trading_value >= 1000000000 # condition 1: Transaction amount of more than 1 billion won 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) # condition 2: Today's stock price change rate is more than 5%
    condition = condition1 & condition2 & condition3
    return condition

b1_lg.condition = sampling1  # Define the condition function directly (sampling1) and store it in the condition property 

### **Define MachinelearnigBuyer objects**
#### Machine learning buyer is responsible for learning artificial intelligence models. All information related to the artificial intelligence model should be included in the properties of the Machine Learning Buyer object.
#### Defining a model is the same as using any package, and you can put the defined model object into the `algorithm` property of the machine learning buyer.

In [ ]:
from lightgbm import LGBMClassifier

# machinelearning_buyer: Object that determines acquisition by machine learning model
b2_lg = sai.MachinelearningBuyer()

# Save user-defined models to algorithm properties
scale_pos_weight = round(72/28 , 2)
params = {  'random_state' : 42,
            'scale_pos_weight' : scale_pos_weight,
            'learning_rate' : 0.1, 
            'num_iterations' : 1000,
            'max_depth' : 4,
            'n_jobs' : 30,
            'boost_from_average' : False,
            'objective' : 'binary' }

b2_lg.algorithm =  LGBMClassifier( **params )

#### Consequently, Conditional Buyers and Machine Learning Buyers, which are sub-objects of Buyers, were defined.
### **Definition of SubSeller object**
#### The SubSeller object only provides a way to resell all of the purchased shares the day after the purchase date. It creates one such SubSeller object.

In [ ]:
# SubSeller: Object that determines selling all of the following days
sell_all = sai.SubSeller() 

### **Trader / Buyer / Seller Object Definitions**
#### Trader objects have `name` attributes to distinguish traders, `label` attributes to set dependent variables, and `buyer` and `seller` attributes to store buyer and seller objects.

In [ ]:
# Trader Object   
t1 = sai.Trader()
t1.name = 'saiLightGBM' # Trader's name
t1.label = 'class&0.02' # Set the Trader dependent variable (do not set if it is regression analysis) 
t1.buyer = sai.Buyer([b1_lg, b2_lg]) # [ conditional buyer, machinelearning buyer ] 
t1.seller = sai.Seller(sell_all)

lst_trader.append(t1)

#### If you look at the label here, it says "class&0.02", which is a **classification** problem, which means **1** if the next day's closing rate of change is more than **0.02**, otherwise **0**. If you don't want to classify it, you can set it as "reg" and use the next-day closing rate (next_change) itself as a dependent variable.
#### When calling a buyer object, the buyer puts the conditional buyer and machine learning buyer defined above in the list, and the seller immediately puts the sub-seller object.
#### Finally, I add a trader to the lst_trader that I first created.

<br>

#### This defined all Trader objects. **(Defining deep learning models has additional properties, so you can refer to the model examples below (ex7, ex8) or find detailed descriptions in the files `02-1.tensorflow_example.ipynb`, `02-2.pytorch_example.ipynb`**

<br>

#### Below is an example of defining several machine learning models.

<br>

### **ex1) LightGBM** 

In [ ]:
from lightgbm import LGBMClassifier

# conditional_buyer: Object that determines acquisition based on data filtering conditions 
b1_lg = sai.ConditionalBuyer()

def sampling1(df): # Create a conditional function
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) # Remove exceptions that exceed upper and lower limits
    condition2 = df.D0_trading_value >= 1000000000 # condition 1: Transaction amount of more than 1 billion won 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) # condition 2: Today's stock price change rate is more than 5%
    condition = condition1 & condition2 & condition3
    return condition

b1_lg.condition = sampling1  # Define the condition function directly (sampling1) and store it in the condition property 


# machinelearning_buyer: Object that determines acquisition by machine learning model
b2_lg = sai.MachinelearningBuyer()

# Save user-defined models to algorithm properties
scale_pos_weight = round(72/28 , 2)
params = {  'random_state' : 42,
            'scale_pos_weight' : scale_pos_weight,
            'learning_rate' : 0.1, 
            'num_iterations' : 1000,
            'max_depth' : 4,
            'n_jobs' : 30,
            'boost_from_average' : False,
            'objective' : 'binary' }

b2_lg.algorithm =  LGBMClassifier( **params )


# SubSeller: Object that determines selling all of the following days
sell_all = sai.SubSeller() 


# Trader Object   
t1 = sai.Trader()
t1.name = 'saiLightGBM' # Trader's name
t1.label = 'class&0.02' # Set the Trader dependent variable (do not set if it is regression analysis) 
t1.buyer = sai.Buyer([b1_lg, b2_lg]) # [ conditional buyer, machinelearning buyer ] 
t1.seller = sai.Seller(sell_all)

lst_trader.append(t1)

<br>

### **ex2) XGBoost** 

In [ ]:
from xgboost import XGBClassifier

b1_xgb = sai.ConditionalBuyer() 

def sampling2(df): 
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_xgb.condition = sampling2


b2_xgb = sai.MachinelearningBuyer()  

scale_pos_weight = round(72/28 , 2)
b2_xgb.algorithm = XGBClassifier(random_state = 42,
                   n_jobs=30,
                   scale_pos_weight=scale_pos_weight,
                   learning_rate=0.1,
                   max_depth=4,
                   n_estimators=1000,
                   )  

sell_all = sai.SubSeller()


t2 = sai.Trader()
t2.name = 'saiXGboost' 
t2.label = 'class&0.02' 
t2.buyer = sai.Buyer([b1_xgb, b2_xgb])
t2.seller = sai.Seller(sell_all) 

lst_trader.append(t2) 

<br>

### **ex3) LogisticRegression** 

In [ ]:
from sklearn.linear_model import LogisticRegression

b1_lr = sai.ConditionalBuyer()

def sampling3(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_lr.condition = sampling3


b2_lr = sai.MachinelearningBuyer()  

b2_lr.algorithm = LogisticRegression()


sell_all = sai.SubSeller() 


t3 = sai.Trader()
t3.name = 'saiLogisticRegression'  
t3.label = 'class&0.02' 
t3.buyer = sai.Buyer([b1_lr, b2_lr]) 
t3.seller = sai.Seller(sell_all)

lst_trader.append(t3) 

<br> 

### **ex4) Support Vector Machine**

In [ ]:
from sklearn.svm import SVC

b1_sv = sai.ConditionalBuyer()

def sampling4(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_sv.condition = sampling4 


b2_sv = sai.MachinelearningBuyer()  

b2_sv.algorithm = SVC() 


sell_all = sai.SubSeller() 


t4 = sai.Trader()
t4.name = 'saiSupportVectorMachine'  
t4.label = 'class&0.02' 
t4.buyer = sai.Buyer([b1_sv, b2_sv]) 
t4.seller = sai.Seller(sell_all)

lst_trader.append(t4) 

<br>

### **ex5) Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

b1_dt = sai.ConditionalBuyer()

def sampling5(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_dt.condition = sampling5 


b2_dt = sai.MachinelearningBuyer()  

b2_dt.algorithm = DecisionTreeClassifier() 


sell_all = sai.SubSeller() 


t5 = sai.Trader()
t5.name = 'saiDecisionTree'  
t5.label = 'class&0.02' 
t5.buyer = sai.Buyer([b1_dt, b2_dt]) 
t5.seller = sai.Seller(sell_all)

lst_trader.append(t5) 

<br>

### **ex6) RandomForest** 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

b1_rf = sai.ConditionalBuyer()

def sampling6(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_rf.condition = sampling6 


b2_rf = sai.MachinelearningBuyer()  

b2_rf.algorithm = RandomForestClassifier() 


sell_all = sai.SubSeller() 


t6 = sai.Trader()
t6.name = 'saiDecisionTree'  
t6.label = 'class&0.02' 
t6.buyer = sai.Buyer([b1_rf, b2_rf]) 
t6.seller = sai.Seller(sell_all)

lst_trader.append(t6) 

----------


<br>

### **Deep learning model**

#### In order to use a deep learning model, a data structure that was two-dimensional must be converted into three-dimensional. Therefore, the attribute `data_transform` is added.
#### Depending on which framework you use, you should include the `framework` attribute.
#### In the case of tensorflow, the hyperparameter or callback function must be put in `params`,
#### In the case of pytorch, in addition to the `params` attribute, the `device` attribute that sets gpu and cpu, or the `optim` attribute that sets the loss function and optimizer is additionally required.

<br>

#### **(More details can be found in the files `02-1.tensorflow_example.ipynb` and `02-2.pytorch_example.ipynb` for these deep learning models.)**

<br>

### **ex7) LSTM - keras**  

In [ ]:
b1_ls = sai.ConditionalBuyer()

def sampling1(df): 
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = (df.D0_Close * df.D0_Volume) >= 100000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3 
    return condition

b1_ls.condition = sampling1

b2_ls = sai.MachinelearningBuyer()

# User-defined functions (users who want deep learning modeling)
def transform1(data): # A function that converts into a two-dimensional structure / data: list (lst_time_series)
    data_2d = []
    n_col = int(len(data[0]) / 10) 
    for row in data:      
        data_2d.append([])
        for i in range(0, len(row), n_col):
            data_2d[-1].append(row[i:i+n_col])
    
    return np.array(data_2d)
    

# Directly define a two-dimensional structure transformation function (transform) and store it in the data_transform property
b2_ls.data_transform = transform1 

from keras.callbacks import EarlyStopping

# deep learning framework 
b2_ls.framework = "tensorflow"

# parameters for model.fit() 
early_stopping = EarlyStopping(monitor='val_loss', patience=7)
b2_ls.params = {
    "epochs": 20, 
    "batch_size": 64,
    "callbacks": [early_stopping]
    }

#model.add(keras.layers.Dropout(0.2))
# defines a model 
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(10, 48)))
model.add(keras.layers.LSTM(128, activation='selu', return_sequences=True))
model.add(keras.layers.LSTM(64, activation='selu', return_sequences=True))
model.add(keras.layers.LSTM(32, activation='selu', return_sequences=False))
model.add(keras.layers.Dense(1, activation='sigmoid'))
    
model.compile(optimizer=keras.optimizers.Adam(
    # learning_rate=keras.optimizers.schedules.ExponentialDecay(0.05,decay_steps=100000,decay_rate=0.96)), 
    learning_rate = 0.001), 
    loss="binary_crossentropy",
    metrics=['accuracy'])

b2_ls.algorithm = model

sell_all = sai.SubSeller() 

t7 = sai.Trader()
t7.name = 'saiLSTM_tf' 
t7.label = 'class&0.02' 
t7.buyer = sai.Buyer([b1_ls, b2_ls]) 
t7.seller = sai.Seller(sell_all)

lst_trader.append(t4)

<br>

### **ex8) LSTM - PyTorch**  

In [ ]:
import torch
import torch.nn as nn
import numpy as np

b1_lspt = sai.ConditionalBuyer()

def sampling5(df): 
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3)
    condition2 = (df.D0_Close * df.D0_Volume) >= 1000000000 
    # condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition4 = (df.D0_CCI <= -100)
    condition = condition1 & condition2 & condition4  
    return condition

b1_lspt.condition = sampling5

b2_lspt = sai.MachinelearningBuyer()

# ⭐ User-defined functions (users who want deep learning modeling)
def transform(data): # A function that converts into a two-dimensional structure / data: list (lst_time_series)
    data_2d = []
    n_col = int(len(data[0]) / 10) 
    for row in data:      
        data_2d.append([])
        for i in range(0, len(row), n_col):
            data_2d[-1].append(row[i:i+n_col])
    
    return np.array(data_2d)
    

# Directly define a two-dimensional structure transformation function (transform) and store it in the data_transform property
b2_lspt.data_transform = transform 

# framework 
b2_lspt.framework = "pytorch"

# devcice  
b2_lspt.device = device 

##### LSTM Model Definition #####
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # hidden state와 cell state의 값을 0으로 초기화 함 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # hidden state 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # cell state 
        
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.fc(out[:, -1, :])
        # out = torch.sigmoid(out) 
                      
        return out
    
                      
# learning rate 
learning_rate = 0.01

# create a model 
input_size, hidden_size, num_layers, output_size = 48, 64, 12, 1 
model = LSTM(input_size, hidden_size, num_layers, output_size)   

b2_lspt.algorithm = model

# hyper parameters for model fitting 
b2_lspt.params = {
    "epochs": 20, 
    "batch_size": 64,
    }


pos_weight = torch.tensor([7.0], device=device)

# loss function & optimizer 
b2_lspt.optim = {
    "criterion": nn.BCEWithLogitsLoss(pos_weight=pos_weight), 
    "optimizer": torch.optim.Adam(model.parameters(), lr=learning_rate)  
    }


sell_all = sai.SubSeller() 


t8 = sai.Trader()
t8.name = 'saiLSTM_pt' 
t8.label = 'class&0.02' 
t8.buyer = sai.Buyer([b1_lspt, b2_lspt]) 
t8.seller = sai.Seller(sell_all)

lst_trader.append(t8)

<br>

#### If you put these eight traders into the lst_trader, you can learn each model at once and run a stock trading simulation.
#### Examples of model learning and stock trading simulations using multiple models can be found in the file `tutorials/03.different_models.ipynb`.